In [12]:
import sys
sys.path.append(r'/home/a/Lwy_research/equity_tail_strategy')
from _functions import *
from update_seconddata_ver2 import *

In [2]:
# 这一部分函数之前有
# 计算完整的秒序列
def calc_sec_list(keep_last = False):
    time_series=[]  #所有的秒序列,不包括早尾盘集合竞价(也就是093000-145659的)
    for i in [9,10,11,13,14]:
        for j in range(60):
            if i==9 and j<30:
                continue
            elif i==11 and j>=30:
                continue
            elif i==14 and j>=57:
                continue
            time_series.extend([int(str(i)+'0'*(2-len(str(j)))+str(j)+'0'*(2-len(str(x)))+str(x)) for x in range(60)])
    if keep_last: # 保留最后一个时点
        time_series += [150000]
    return time_series

# 转化为1秒级时间,比如如果time为93524380，且转化的1s时间为93524000
def get_second_1s(time_10ms):
    div = 1000
    return round(round(time_10ms // div) * div) # 先取整再标准化为Level2时间

# 转化为分钟时间,比如如果time为93524380，且转化的1s时间为93500000
def get_second_60s(time_10ms):
    div = 100000
    return round(round(time_10ms // div) * div) # 先取整再标准化为Level2时间

In [3]:
time_step = "60s" # 这里time_step有可能是1s、5s、10s、60s，这里用60s只作为示例和核对作用
get_second_map = eval(rf"get_second_{time_step}")
second_list = calc_sec_list(keep_last=True)
second_list = [x*1000 for x in second_list]
standard_time_list = []
for time in tqdm(second_list):
    if get_second_map(time) not in standard_time_list:
        standard_time_list.append(get_second_map(time))
len(standard_time_list)

100%|██████████| 14221/14221 [00:00<00:00, 1107346.09it/s]


238

In [21]:
date = "20250320"
sample_order_data = pd.read_feather(rf"./example_data/sample_order_{date}.fea")
sample_trans_data = pd.read_feather(rf"./example_data/sample_trans_{date}.fea")
order_data_dict = change_df2dict(sample_order_data)
trans_data_dict = change_df2dict(sample_trans_data)
intra_support_data =  read_daily_support(date)
intra_support_data.index = ['%06d'%int(x) for x in intra_support_data.index]
code_list = set(order_data_dict.keys()) & set(order_data_dict.keys()) & set(intra_support_data.index)



In [23]:
def calc_stock_second_data_ver2(code, trans_data, order_data,  intra_support_data, standard_time_list, get_second_map):
    '''
    计算一些逐笔成交、逐笔委托构造的进阶指标
    '''
    tot_fac_num = 0
    second_data_list = []
    for func in [calc_bft_ver4d1, calc_bft_ver5d1, calc_time_dist, calc_trans_abnormal, calc_wait_time1, calc_fjl1, calc_bhv]:
        try:
            fac_res = func(code, trans_data, order_data, intra_support_data, standard_time_list, get_second_map)
            
            tot_fac_num += fac_res.shape[1]
            second_data_list.append(fac_res)
        except Exception as e:
            print(code,e)
            
    second_data = pd.concat(second_data_list, axis = 1)
    assert len(second_data.columns) == tot_fac_num
    second_data = second_data.reindex(standard_time_list)
    second_data.index.name = "second"
    second_data["Code"] = code
    second_data = second_data.reset_index().set_index("Code")
# try:


    return second_data


# 这里intra_support_data仅仅作用为提供了一个preclose，实盘直接用查到的preclose就行
second_data_list = Parallel(n_jobs=32)(delayed(calc_stock_second_data_ver2)(code, trans_data_dict[code], order_data_dict[code],  intra_support_data, standard_time_list, get_second_map) for code in tqdm(code_list))
second_data = pd.concat(second_data_list)

100%|██████████| 997/997 [01:45<00:00,  9.42it/s]


In [28]:
second_data.reset_index().to_feather(rf"./example_data/sample_secondv2_{date}.fea", compression ="zstd")

In [30]:
intra_support_data.reset_index().to_csv(rf"./example_data/sample_intra_support_{date}.csv")